# Introducción: Proyecto de Machine Learning Parte 1

Al leer un libro de ciencia de datos o seguir un curso, puedes sentir que tienes piezas individuales, pero no sabes cómo unirlas. Dar el siguiente paso y resolver un problema de machine learning completo puede ser desalentador, pero enfrentarse y completar un primer proyecto le dará la confianza para afrontar cualquier problema relacionado con data science. Esta serie de notebooks analizará una solución completa de machine learning con un conjunto de datos del mundo real para que pueda ver cómo se unen todas las piezas.

Seguiremos paso a paso el flujo general de trabajo de machine learning:

1. Limpieza y formateo de datos.
2. Análisis exploratorio de datos
3. Ingeniería y selección de características.
4. Comparar varios modelos de machine learning en una métrica de rendimiento.
5. Ajuste del hiperparámetro en el mejor modelo
6. Evaluar el mejor modelo en el conjunto de pruebas
7. Interpretar los resultados del modelo.
8. Sacar conclusiones y documentar el trabajo.

En el camino veremos cómo cada paso fluye hacia el siguiente y cómo implementar específicamente cada parte en Python.

Definir esta estructura antes de tiempo nos permite ver cómo un paso fluye hacia el otro. Sin embargo, el proceso de machine learning es un procedimiento iterativo, por lo que no siempre seguimos estos pasos de manera lineal. Es posible que volvamos a revisar un paso anterior en función de los resultados de una etapa posterior. Por ejemplo, si bien podemos realizar una selección de características antes de construir cualquier modelo, podemos usar los resultados del modelado para retroceder y seleccionar un conjunto diferente de características. O el modelado puede generar resultados inesperados que signifiquen que queremos explorar nuestros datos desde otro ángulo.

A lo largo de esta serie de notebooks, el objetivo es mostrar cómo todas las diferentes prácticas de data science se unen para formar un proyecto completo. Trato de centrarme más en la implementación de los métodos en lugar de explicarlos en un nivel bajo, pero he proporcionado recursos para aquellos que desean profundizar. Uno de los mejores libros individuales para aprender lo básico e implementar prácticas de machine learning en Python es [Hands-On Machine Learning with Scikit-Learn and Tensorflow](http://shop.oreilly.com/product/0636920052289.do) de [Aurelion Geron](https://twitter.com/aureliengeron?lang=en).

## Definición del problema

El primer paso es comprender el problema que estamos tratando de resolver y los datos disponibles. En este proyecto, trabajaremos con [datos de energía de edificios disponibles públicamente de la ciudad de Nueva York](http://www.nyc.gov/html/gbee/html/plan/ll84_scores.shtml).

**El objetivo es utilizar los datos de energía para construir un modelo que pueda predecir la puntuación Energy Star de un edificio e interpretar los resultados para encontrar los factores que influyen en dicha puntuación**.

Los datos incluyen la puntuación Energy Star, por tanto, se trata de una tarea de machine learning de regresión supervisada:

+ **Supervisado**: tenemos acceso tanto a las características como al objetivo y nuestro objetivo es entrenar un modelo que pueda aprender un mapeo entre los dos.
+ **Regresión**: la puntuación de Energy Star es una variable continua (en este caso va de 0 a 100).

Queremos desarrollar un modelo que sea a la vez **preciso**: puede predecir el puntaje Energy Star cerca del valor real, e **interpretable**: podemos entender las predicciones del modelo. Una vez que conozcamos el objetivo, podemos usarlo para guiar nuestras decisiones a medida que profundizamos en los datos y construimos modelos. Durante el entrenamiento, queremos que el modelo aprenda la relación entre las características y la puntuación, por lo que le damos las características y la respuesta. Luego, para probar cómo de bien ha aprendido el modelo, ¡lo evaluamos en un conjunto de pruebas donde nunca ha visto las respuestas!

## Imports

Usaremos las bibliotecas estándar de ciencia de datos y machine learning: numpy, pandas y scikit-learn. También utilizamos matplotlib y seaborn para visualización.

In [2]:
# Pandas y numpy para manipulación de datos
import pandas as pd
import numpy as np

# Quitamos los warnings sobre el establecimiento de un valor en una copia de un slice
pd.options.mode.chained_assignment = None

# Mostramos hasta 60 columnas de un dataframe
pd.set_option('display.max_columns', 60)

# Visualización Matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Establece el tamaño de fuente por defecto
plt.rcParams['font.size'] = 24

# Herramienta interna ipython para establecer el tamaño de la figura
from IPython.core.pylabtools import figsize

# Visualizción por Seaborn
import seaborn as sns
sns.set(font_scale = 2)

# División de los datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split

## Limpieza de datos

Contrariamente a lo que pueda parecer, no todos los conjuntos de datos son un grupo de observaciones perfectamente saneado, sin valores perdidos ni anomalías (como puede creerse observando los conjuntos de datos de [mtcars](http://stat.ethz.ch/R-manual/R-devel/library/datasets/html/mtcars.html) e [iris](https://archive.ics.uci.edu/ml/datasets/iris)). Los datos del mundo real son confusos, lo que significa que debemos [limpiarlos y cambiarlos](https://www.springboard.com/blog/data-wrangling/) a un formato aceptable antes de que podamos comenzar el análisis. La limpieza de datos es una parte poco glamurosa pero necesaria de la mayoría de los problemas reales de data science.

Cargaremos nuestros datos en un `dataframe` de pandas, una de las estructuras de datos más útiles de data science. Piense en ello como una hoja de cálculo dentro de Python que podemos manipular, limpiar y visualizar fácilmente. Pandas [tiene muchos métodos](http://pandas.pydata.org/pandas-docs/stable/) para ayudar a que el flujo de información de ciencia de datos / machine learning sea lo más sencillo posible.